Import necessary libraries.

In [1]:
import pandas as pd
from sqlalchemy import create_engine

Run queries to get selected data from SQL into Pandas dataframe. 

In [2]:
engine = create_engine('sqlite:///D:\\Data\\DuvalProperties.sqlite')
con = engine.connect()
inv = con.execute(
    """
    SELECT
        RE, section, township, range, tile, mailing_address_1, mailing_address_2, city AS mail_city, 
        state AS mail_state, substr(zipcode, 1, 5) AS mail_zipcode, 
		subdivision_name, neighborhood, perc_capped, just_value, 
        ROUND((school_taxable - just_value) / just_value * 100,2) AS perc_school_deduction, 
		ROUND((county_taxable - just_value) / just_value * 100,2) AS perc_county_deduction, 
        tax_district, lot_sf,
		building, type_descr, style, class, quality, actual_yr_built, 
        date('now') - actual_yr_built AS age, effec_yr_built, date('now') - effec_yr_built AS effec_age, 
        value AS building_value, heated_sf
    FROM Parcel
    LEFT JOIN Building
    USING (RE)
    WHERE property_use = '0100' AND type_descr LIKE '%SFR%'
    """
    )
housing_inventory = pd.DataFrame(inv.fetchall(),
                                columns=['RE', 'section', 'township', 'range', 'tile', 'mailing_address_1',
                                        'mailing_address_2', 'mail_city', 'mail_state', 'mail_zipcode',
                                        'subdvision_name', 'neighborhood', 'perc_capped', 'just_value',
                                        'perc_school_deduction', 'perc_county_deduction', 'tax_district',
                                        'lot_sf', 'building', 'type_descr', 'style', 'class', 'quality', 
                                        'actual_yr_built', 'age', 'effec_yr_built', 'effec_age',
                                        'building_value', 'heated_sf'])
fts = con.execute(
    """
    SELECT
        RE, building, ft_descr, feature_count, ft_depr_value
    FROM Parcel
    LEFT JOIN Building
    USING (RE)
    LEFT JOIN Feature
    USING (RE, building)
    LEFT JOIN (
    SELECT RE, building, COUNT(ft_descr) AS feature_count, IFNULL(SUM(deprec_value),0) AS ft_depr_value
    FROM Parcel
    LEFT JOIN Building 
    USING (RE)
    LEFT JOIN Feature
    USING (RE, building)
    WHERE property_use = '0100' AND type_descr LIKE '%SFR%'
    GROUP BY RE, building) USING (RE, building)
    WHERE property_use = '0100' AND type_descr LIKE '%SFR%'
    GROUP BY RE,building,ft_descr
    """
    )
housing_features = pd.DataFrame(fts.fetchall(),
                               columns=['RE', 'building', 'ft_descr', 'ft_count', 'ft_depr_val'])

rms = con.execute(
    """
    SELECT RE, building, baths, bedrooms, stories, rooms
    FROM Parcel
    LEFT JOIN Building
    USING (RE)
    LEFT JOIN
    (SELECT  
        RE, building, SUM(baths) AS baths, SUM(bedrooms) AS bedrooms, SUM(stories) AS stories, 
        SUM(rooms) AS rooms
    FROM 
        (SELECT	
            RE, building,
            CASE WHEN structure_descr = 'Baths' THEN units
                ELSE 0 END AS baths,
            CASE WHEN structure_descr = 'Bedrooms' THEN units
                ELSE 0 END AS bedrooms,
            CASE WHEN structure_descr = 'Stories' THEN units
                ELSE 0 END AS stories,
            CASE WHEN structure_descr = 'Rooms / Units' THEN units
                ELSE 0 END AS rooms
        FROM Utility)
        GROUP BY RE, building)
    USING (RE, building)
    WHERE property_use = '0100' AND type_descr LIKE '%SFR%'
    """
    )
housing_rooms = pd.DataFrame(rms.fetchall(),
                               columns=['RE', 'building', 'baths', 'bedrooms', 'stories',
                                       'rooms'])

dists = con.execute(
    """
    SELECT
        RE, char_descr AS district
    FROM Character
    LEFT JOIN Building
    USING (RE)
    LEFT JOIN Parcel
    USING (RE)
    WHERE property_use = '0100' AND type_descr LIKE '%SFR%'
    """
    )
districts = pd.DataFrame(dists.fetchall(),
                               columns=['RE', 'district'])

land = con.execute(
    """
    SELECT
        RE, use_descr AS land_use, land_val
    FROM Parcel
    LEFT JOIN Building
    USING (RE)
    LEFT JOIN Common
    USING (RE)
    WHERE property_use = '0100' AND type_descr LIKE '%SFR%'
    """
    )
land_uses = pd.DataFrame(land.fetchall(),
                               columns=['RE', 'land_use', 'land_val'])

sub = con.execute(
    """
    SELECT
        RE, building, substructure_descr, sub_ft_count, sub_ft_effec_area
    FROM Parcel
    LEFT JOIN Building
    USING (RE)
    LEFT JOIN Subarea
    USING (RE, building)
    LEFT JOIN (
    SELECT RE, building, COUNT(substructure_descr) AS sub_ft_count, IFNULL(SUM(Subarea.effec_area),0) AS sub_ft_effec_area
    FROM Parcel
    LEFT JOIN Building 
    USING (RE)
    LEFT JOIN Subarea
    USING (RE, building)
    WHERE property_use = '0100' AND type_descr LIKE '%SFR%' AND substructure_descr <> 'Base Area'
    GROUP BY RE, building) USING (RE, building)
    WHERE property_use = '0100' AND type_descr LIKE '%SFR%' AND substructure_descr <> 'Base Area'
    GROUP BY RE, building, substructure_descr
    """
    )
subarea = pd.DataFrame(sub.fetchall(),
                               columns=['RE', 'building', 'substructure_descr', 'sub_ft_count', 
                                        'sub_ft_effec_area'])

site = con.execute(
    """
    SELECT
        Parcel.RE, street_num, direction, street_name, street_type, unit, Site.city, Site.zipcode, 
        building_num
    FROM Parcel
    LEFT JOIN Building
    USING (RE)
    LEFT JOIN Site
    ON Parcel.RE = Site.RE AND Building.building = Site.building_num
    WHERE property_use = '0100' AND type_descr LIKE '%SFR%'
    GROUP BY Parcel.RE, Building.building
    """
    )
sites = pd.DataFrame(site.fetchall(),
                               columns=['RE', 'street_num', 'direction', 'street_name', 'street_type',
                                       'unit', 'city', 'zipcode', 'building'])

top_sales = con.execute(
    """
    SELECT RE, MAX(price) AS max_price, sale_date AS max_date
    FROM Sale
    LEFT JOIN Qualification 
    ON Sale.qualification = Qualification.q_id
    LEFT JOIN Parcel
    USING (RE)
    LEFT JOIN Building
    USING (RE)
    WHERE status = 'Qualified' AND price > 10000 AND improved = 'I' AND property_use = '0100' 
    AND type_descr LIKE '%SFR%'
    GROUP BY RE
    """
    )
max_sales = pd.DataFrame(top_sales.fetchall(),
                               columns=['RE', 'max_price', 'max_date'])

recent = con.execute(
    """
    SELECT RE, MAX(sale_date) AS most_recent_date, price AS most_recent_price
    FROM Sale
    LEFT JOIN Qualification 
    ON Sale.qualification = Qualification.q_id
    LEFT JOIN Parcel
    USING (RE)
    LEFT JOIN Building
    USING (RE)
    WHERE status = 'Qualified' AND price > 10000 AND improved = 'I' AND property_use = '0100' 
    AND type_descr LIKE '%SFR%'
    GROUP BY RE
    """
    )
recent_sales = pd.DataFrame(recent.fetchall(),
                               columns=['RE', 'most_recent_date', 'most_recent_price'])

sales = con.execute(
    """
    SELECT RE, COUNT(DISTINCT(or_bk_pg)) AS q_sales
    FROM Sale
    LEFT JOIN Qualification 
    ON Sale.qualification = Qualification.q_id
    LEFT JOIN Parcel
    USING (RE)
    LEFT JOIN Building
    USING (RE)
    WHERE status = 'Qualified' AND price > 10000 AND improved = 'I' AND property_use = '0100' 
    AND type_descr LIKE '%SFR%'
    GROUP BY RE 
    """
    )
all_sales = pd.DataFrame(sales.fetchall(),
                               columns=['RE', 'q_sales'])

sales_3yrs = con.execute(
    """
    SELECT RE, COUNT(DISTINCT(or_bk_pg)) AS q_sales
    FROM Sale
    LEFT JOIN Qualification 
    ON Sale.qualification = Qualification.q_id
    LEFT JOIN Parcel
    USING (RE)
    LEFT JOIN Building
    USING (RE)
    WHERE status = 'Qualified' AND price > 10000 AND improved = 'I' AND property_use = '0100' 
    AND type_descr LIKE '%SFR%' AND sale_date > '2017-12-31'
    GROUP BY RE
    """
    )
all_sales_3yrs = pd.DataFrame(sales_3yrs.fetchall(),
                               columns=['RE', 'q_sales_3yrs'])

first_seller = con.execute(
    """
    SELECT RE, seller
    FROM (
    SELECT RE, seller, MIN(sale_id)
    FROM Sale
    LEFT JOIN Qualification 
    ON Qualification.q_id = Sale.qualification
    WHERE improved = 'I' AND (status = 'Qualified' OR status = 'Excluded')
    GROUP BY RE)
    """
    )
first_sellers = pd.DataFrame(first_seller.fetchall(),
                               columns=['RE', 'first_seller'])

con.close()

In [3]:
def most_common(df, col_name):
    """Gets 10 most common categorical variables in dataframe column. Changes all
    other variables to 'Other'.

    Parameters
    ----------
    df : DataFrame
        Pandas dataframe
    col_name : str
        Name of column in dataframe

    Returns
    -------
    DataFrame
        Pandas series with variables outside of 10 most common changed to 'Other'
    """
    counts = df[col_name].value_counts()
    top_10 = list(counts[:10].index)
    new_col = []
    for thing in df[col_name]:
        if thing in top_10:
            new_col.append(thing)
        else:
            new_col.append(None)
    df[col_name] = new_col
    print(df[col_name].value_counts())

In [4]:
def get_dummies(df, col_name, prefix, groupby_list):
    """Gets dummy variables of 10 most common variables and 'Other'

    Parameters
    ----------
    df : DataFrame
        Pandas dataframe
    col_name : str
        Name of column in dataframe
    prefix : str
        Prefix used in dummy variable column names
    groupby_list : list
        List of column names to group df by before getting sums for merged dataframe

    Returns
    -------
    DataFrame
        Pandas series with variables outside of 10 most common changed to 'Other'
    """
    most_common(df, col_name)
    df2 = pd.get_dummies(df, columns=[col_name], prefix=prefix)
    df2 = df2.groupby(by=groupby_list).sum().reset_index()
    return df2

In [5]:
housing_features = get_dummies(housing_features, 'ft_descr', 'ft', ['RE', 'building'])

Firep Prf     74498
Pool          29482
Firep Ms      29256
Scr Porch     26670
Cov Patio     19699
Deck Wd       18924
Shed wood     12275
Firep Gas     12037
Screen En     11080
Carport Al    10602
Name: ft_descr, dtype: int64


In [6]:
districts_final = get_dummies(districts, 'district', 'd', ['RE'])

LAKE                                          4788
Local Riverside Avondale Historic District    3814
ST JOHNS RIVER                                1671
GOLF COURSE                                   1223
Local Springfield Historic District           1221
CANAL                                         1190
MISCELLANEOUS WATERWAY                         770
TROUT RIVER                                    531
National Register Ortega Historic District     446
OCEAN FRONT                                    317
Name: district, dtype: int64


In [7]:
landuses_final = get_dummies(land_uses, 'land_use', 'land', ['RE'])

RES LD 3-7 UNITS PER AC             180321
RES POND LD 3-7 UNITS PER AC         21400
RES MD 8-19 UNITS PER AC             16478
RES NATURAL LD 3-7 UNITS PER AC      14453
RES RURAL 2 OR LESS UNITS PER AC      5761
RES RIVER LD 3-7 UNITS PER AC         3452
RES CANAL LD 3-7 UNITS PER AC         3212
RES GOLF LD 3-7 UNITS PER AC          2089
RES MARSH LD 3-7 UNITS PER AC         1185
COMM/RES/OFF                          1160
Name: land_use, dtype: int64


In [8]:
subarea_final = get_dummies(subarea, 'substructure_descr', 'sub', ['RE','building'])

Finished Open Porch       213666
Finished Garage           138782
Addition                   71115
Finished upper story 1     42871
Fin Screened Porch         40073
Unfinished Storage         38047
Unfinished Garage          23556
Unfin Open Porch           22897
Finished Encl Porch        19477
Finished Carport           15973
Name: substructure_descr, dtype: int64


In [11]:
df = pd.merge(housing_inventory, housing_features, on=['RE', 'building'])
df = df.merge(housing_rooms, on=['RE', 'building'])
df = df.merge(districts_final, how='left', on='RE')
df = df.merge(landuses_final, on='RE')
df = df.merge(subarea_final, how='left', on=['RE','building'])
df = df.merge(sites, how='left', on=['RE', 'building'])
df = df.merge(max_sales, how='left', on='RE')
df = df.merge(recent_sales, how='left', on='RE')
df = df.merge(all_sales, how='left', on='RE')
df = df.merge(all_sales_3yrs, how='left', on='RE')
df = df.merge(first_sellers, how='left', on='RE')
df.head()

,RE,section,township,range,tile,mailing_address_1,mailing_address_2,mail_city,mail_state,mail_zipcode,...,unit,city,zipcode,max_price,max_date,most_recent_date,most_recent_price,q_sales,q_sales_3yrs,first_seller
0,0000060030R,01,2S,23E,3401,2503 SUMMERFIELD LN,None,JACKSONVILLE,FL,32234,...,None,JACKSONVILLE,32234,192000.0,2004-07-09,2004-07-09,192000.0,1.0,NaN,HIGHSMITH EARL O &LINDA K
1,0000070010R,01,2S,23E,3401,C/O TAX DEPARTMENT,9540 SAN JOSE BLVD,JACKSONVILLE,FL,32257,...,None,JACKSONVILLE,32234,150000.0,2018-01-04,2018-01-04,150000.0,2.0,1.0,
2,0000090100R,12,2S,23E,3412,2495 U S 301 HWY N,None,BALDWIN,FL,32234,...,None,JACKSONVILLE,32234,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0000110000R,12,2S,23E,3412,2425 US HWY 301 N,None,BALDWIN,FL,32234,...,None,JACKSONVILLE,32234,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0000120010R,12,2S,23E,3412,2204 US HIGHWAY 301 N,None,JACKSONVILLE,FL,32234,...,None,JACKSONVILLE,32234,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df.shape

(252718, 91)

In [13]:
df.to_csv('D:/Data/JacksonvilleHouses.csv')

In [14]:
engine = create_engine('sqlite:///D:\\Data\\DuvalProperties.sqlite')
con = engine.connect()
sale_db = con.execute(
    """
    SELECT RE, sale_id, seller, or_bk_pg AS trans_id, sale_date, price, status
    FROM Sale
    LEFT JOIN qualification
    ON Sale.qualification = Qualification.q_id
    LEFT JOIN Parcel
    USING (RE)
    LEFT JOIN Building
    USING (RE)
    WHERE status = 'Qualified' AND price > 10000 AND improved = 'I' AND property_use = '0100' 
    AND type_descr LIKE '%SFR%'
    GROUP BY trans_id, sale_date, RE
    """
    )
house_sales = pd.DataFrame(sale_db.fetchall(),
                                columns=['RE', 'sale_id', 'seller', 'trans_id', 'sale_date', 'price',
                                        'status'])
con.close()
house_sales.to_csv('D:/Data/JacksonvilleHomeSales.csv')